**Copyright 2023 The TF-Agents Authors.**

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# SAC minitaur com a API Actor-Learner

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/agents/tutorials/7_SAC_minitaur_tutorial">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/agents/tutorials/7_SAC_minitaur_tutorial.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/agents/tutorials/7_SAC_minitaur_tutorial.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/agents/tutorials/7_SAC_minitaur_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>


## Introdução

Este exemplo mostra como treinar um agente [Soft Actor Critic](https://arxiv.org/abs/1812.05905) no ambiente [Minitaur](https://github.com/bulletphysics/bullet3/blob/master/examples/pybullet/gym/pybullet_envs/bullet/minitaur.py).

Se você seguiu o [Colab do DQN](https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/agents/tutorials/1_dqn_tutorial.ipynb), isto parecerá bastante familiar. Algumas alterações importantes:

- Mudança do agente de DQN para SAC.
- Treinamento no Minitaur, que é um ambiente muito mais complexo do que o CartPole. O ambiente Minitaur busca treinar um robô quadrúpede para se mover para frente.
- Uso da API Actor-Learner do TF-Agents para o aprendizado por reforço distribuído.

A API é compatível tanto com a coleta de dados distribuída usando um buffer de replay de experiência e um container de variável (servidor de parâmetro) como o treinamento distribuído em vários dispositivos. A API foi criada para ser bastante simples e modular. Utilizamos o [Reverb](https://deepmind.com/research/open-source/Reverb) para o buffer de replay e o container de variável e a [API TF DistributionStrategy](https://www.tensorflow.org/guide/distributed_training) para o treinamento distribuído nas GPUs e TPUs.

Se você ainda não instalou as seguintes dependências, execute:

In [ ]:
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg
!pip install 'imageio==2.4.0'
!pip install matplotlib
!pip install tf-agents[reverb]
!pip install pybullet

## Configuração

Primeiro, vamos importar as diferentes ferramentas necessárias.

In [ ]:
import base64
import imageio
import IPython
import matplotlib.pyplot as plt
import os
import reverb
import tempfile
import PIL.Image

import tensorflow as tf

from tf_agents.agents.ddpg import critic_network
from tf_agents.agents.sac import sac_agent
from tf_agents.agents.sac import tanh_normal_projection_network
from tf_agents.environments import suite_pybullet
from tf_agents.metrics import py_metrics
from tf_agents.networks import actor_distribution_network
from tf_agents.policies import greedy_policy
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_py_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.train import actor
from tf_agents.train import learner
from tf_agents.train import triggers
from tf_agents.train.utils import spec_utils
from tf_agents.train.utils import strategy_utils
from tf_agents.train.utils import train_utils

tempdir = tempfile.gettempdir()

## Hiperparâmetros

In [ ]:
env_name = "MinitaurBulletEnv-v0" # @param {type:"string"}

# Use "num_iterations = 1e6" for better results (2 hrs)
# 1e5 is just so this doesn't take too long (1 hr)
num_iterations = 100000 # @param {type:"integer"}

initial_collect_steps = 10000 # @param {type:"integer"}
collect_steps_per_iteration = 1 # @param {type:"integer"}
replay_buffer_capacity = 10000 # @param {type:"integer"}

batch_size = 256 # @param {type:"integer"}

critic_learning_rate = 3e-4 # @param {type:"number"}
actor_learning_rate = 3e-4 # @param {type:"number"}
alpha_learning_rate = 3e-4 # @param {type:"number"}
target_update_tau = 0.005 # @param {type:"number"}
target_update_period = 1 # @param {type:"number"}
gamma = 0.99 # @param {type:"number"}
reward_scale_factor = 1.0 # @param {type:"number"}

actor_fc_layer_params = (256, 256)
critic_joint_fc_layer_params = (256, 256)

log_interval = 5000 # @param {type:"integer"}

num_eval_episodes = 20 # @param {type:"integer"}
eval_interval = 10000 # @param {type:"integer"}

policy_save_interval = 5000 # @param {type:"integer"}

## Ambiente

Os ambientes no RL representam a tarefa ou o problema que estamos tentando resolver. Os ambientes padrão podem ser facilmente criados no TF-Agents usando `suites`. Temos `suites` diferentes para carregar ambientes de origens como OpenAI Gym, Atari, DM Control, etc., considerando um nome de ambiente de string.

Agora, vamos carregar o ambiente Minitaur a partir da suíte Pybullet.

In [ ]:
env = suite_pybullet.load(env_name)
env.reset()
PIL.Image.fromarray(env.render())

Nesse ambiente, a meta para o agente é treinar uma política que controle o robô Minitaur e o faça avançar o mais rápido possível. Os episódios duram 1000 passos, e o retorno será a soma das recompensas durante o episódio.

Vamos conferir as informações fornecidas pelo ambiente como uma `observation` que a política usará para gerar `actions`.

In [ ]:
print('Observation Spec:')
print(env.time_step_spec().observation)
print('Action Spec:')
print(env.action_spec())

A observação é bastante complexa. Recebemos 28 valores que representam os ângulos, as velocidades e os torques de todos os motores. Em troca, o ambiente espera 8 valores para as ações entre `[-1, 1]`. Esses são os ângulos de motor desejados.

Geralmente, criamos dois ambientes: um para a coleta de dados durante o treinamento e outro para a avaliação. Os ambientes são escritos em python puro e usam arrays do numpy, que a API Actor Learner consome diretamente.

In [ ]:
collect_env = suite_pybullet.load(env_name)
eval_env = suite_pybullet.load(env_name)

## Estratégia de distribuição

Usamos a API DistributionStrategy para permitir a computação do passo de treinamento em vários dispositivos, como diversas GPUs ou TPUs que usam o paralelismo de dados. O passo de treinamento:

- Recebe um lote de dados de treinamento
- Divide em vários dispositivos
- Computa o passo para frente
- Agrega e computa a MÉDIA da perda
- Computa o passo para trás e realiza uma atualização de variável gradiente

Com as APIs DistributionStrategy e Learner do TF-Agents, é bastante fácil trocar entre a execução do passo de treinamento em GPUs (usando MirroredStrategy) para TPUs (usando TPUStrategy) sem mudar nada da lógica de treinamento abaixo.

### Ative a GPU

Se você quiser tentar executar em uma GPU, primeiro precisará ativar as GPUs para o notebook.

- Acesse Edit (Editar)→Notebook Settings (Configurações do notebook)
- Selecione GPU no menu suspenso Hardware Accelerator (Acelerador de hardware)

### Escolha a estratégia

Usamos `strategy_utils` para gerar uma estratégia. Nos bastidores, passar o parâmetro:

- `use_gpu = False` retorna `tf.distribute.get_strategy()`, que usa a CPU
- `use_gpu = True` retorna `tf.distribute.MirroredStrategy()`, que usa todas as GPUs visíveis ao TensorFlow em uma máquina

In [ ]:
use_gpu = True #@param {type:"boolean"}

strategy = strategy_utils.get_strategy(tpu=False, use_gpu=use_gpu)

Todas as variáveis e agentes precisam ser criados em `strategy.scope()`, como você verá abaixo.

## Agente

Para criar um agente SAC, primeiro precisamos criar as redes que ele treinará. O SAC é um agente actor-critic, então vamos precisar de duas redes.

O critic dará estimativas de valores para `Q(s,a)`. Ou seja, ele receberá como entrada uma observação e uma ação e dará uma estimativa do desempenho dessa ação para o estado específico.


In [ ]:
observation_spec, action_spec, time_step_spec = (
      spec_utils.get_tensor_specs(collect_env))

with strategy.scope():
  critic_net = critic_network.CriticNetwork(
        (observation_spec, action_spec),
        observation_fc_layer_params=None,
        action_fc_layer_params=None,
        joint_fc_layer_params=critic_joint_fc_layer_params,
        kernel_initializer='glorot_uniform',
        last_kernel_initializer='glorot_uniform')

Vamos usar esse critic para treinar uma rede `actor`, que permitirá gerar ações a partir de uma observação.

A `ActorNetwork` preverá parâmetros para uma distribuição [MultivariateNormalDiag](https://www.tensorflow.org/probability/api_docs/python/tfp/distributions/MultivariateNormalDiag) com a tangente hiperbólica achatada. Essa distribuição será usada como amostra e condicionada na observação atual sempre que for necessário gerar ações.

In [ ]:
with strategy.scope():
  actor_net = actor_distribution_network.ActorDistributionNetwork(
      observation_spec,
      action_spec,
      fc_layer_params=actor_fc_layer_params,
      continuous_projection_net=(
          tanh_normal_projection_network.TanhNormalProjectionNetwork))

Com essas redes disponíveis, agora podemos instanciar o agente.


In [ ]:
with strategy.scope():
  train_step = train_utils.create_train_step()

  tf_agent = sac_agent.SacAgent(
        time_step_spec,
        action_spec,
        actor_network=actor_net,
        critic_network=critic_net,
        actor_optimizer=tf.keras.optimizers.Adam(
            learning_rate=actor_learning_rate),
        critic_optimizer=tf.keras.optimizers.Adam(
            learning_rate=critic_learning_rate),
        alpha_optimizer=tf.keras.optimizers.Adam(
            learning_rate=alpha_learning_rate),
        target_update_tau=target_update_tau,
        target_update_period=target_update_period,
        td_errors_loss_fn=tf.math.squared_difference,
        gamma=gamma,
        reward_scale_factor=reward_scale_factor,
        train_step_counter=train_step)

  tf_agent.initialize()

## Buffer de replay

Para rastrear os dados coletados do ambiente, vamos usar o [Reverb](https://deepmind.com/research/open-source/Reverb), um sistema de replay eficiente, extensível e fácil de usar do Deepmind. Ele armazena dados de experiências coletados por Actors e consumidos pelo Learner durante o treinamento.

Neste tutorial, isso é menos importante do que o `max_size`, mas, em uma configuração distribuída com treinamento e coleta assíncronos, você deve fazer testes com `rate_limiters.SampleToInsertRatio` usando um samples_per_insert entre 2 e 1000. Por exemplo:

```
rate_limiter=reverb.rate_limiters.SampleToInsertRatio(samples_per_insert=3.0, min_size_to_sample=3, error_buffer=3.0)
```


In [ ]:
table_name = 'uniform_table'
table = reverb.Table(
    table_name,
    max_size=replay_buffer_capacity,
    sampler=reverb.selectors.Uniform(),
    remover=reverb.selectors.Fifo(),
    rate_limiter=reverb.rate_limiters.MinSize(1))

reverb_server = reverb.Server([table])

O buffer de replay é construído usando as especificações que descrevem os tensores a serem armazenados, que podem ser obtidas do agente usando `tf_agent.collect_data_spec`.

Como o agente SAC precisa tanto da observação atual quanto da próxima para computar a perda, definimos `sequence_length=2`.

In [ ]:
reverb_replay = reverb_replay_buffer.ReverbReplayBuffer(
    tf_agent.collect_data_spec,
    sequence_length=2,
    table_name=table_name,
    local_server=reverb_server)

Agora geramos um dataset do TensorFlow a partir do buffer de replay do Reverb. Vamos passar isso para o Learner criar uma amostra das experiências para o treinamento.

In [ ]:
dataset = reverb_replay.as_dataset(
      sample_batch_size=batch_size, num_steps=2).prefetch(50)
experience_dataset_fn = lambda: dataset

## Políticas

No TF-Agents, as políticas representam a noção padrão das políticas no RL: a partir de um `time_step`, é produzida uma ação ou distribuição sobre ações. O principal método é `policy_step = policy.step(time_step)`, em que `policy_step` é uma tupla nomeada `PolicyStep(action, state, info)`. A `policy_step.action` é a `action` que será aplicada ao ambiente, o `state` representa o estado das políticas stateful (RNN), e a `info` pode conter informações auxiliares, como probabilidades log de ações.

Os agentes contêm duas políticas:

- `agent.policy` — a principal política que é usada para avaliação e implantação.
- `agent.collect_policy` — uma segunda política que é usada para coleta de dados.

In [ ]:
tf_eval_policy = tf_agent.policy
eval_policy = py_tf_eager_policy.PyTFEagerPolicy(
  tf_eval_policy, use_tf_function=True)

In [ ]:
tf_collect_policy = tf_agent.collect_policy
collect_policy = py_tf_eager_policy.PyTFEagerPolicy(
  tf_collect_policy, use_tf_function=True)

As políticas podem ser criadas independentemente dos agentes. Por exemplo, use `tf_agents.policies.random_tf_policy` para criar uma política que seleciona aleatoriamente uma ação para cada time_step.

In [ ]:
random_policy = random_py_policy.RandomPyPolicy(
  collect_env.time_step_spec(), collect_env.action_spec())

## Actors

O actor gerencia as interações entre uma política e um ambiente.

- Os componentes Actor contêm uma instância do ambiente (como `py_environment`) e uma cópia das variáveis da política.
- Cada worker Actor executa uma sequência de passos de coleta de dados com os valores locais das variáveis da política.
- As variáveis são atualizadas explicitamente usando a instância do cliente do container da variável no script de treinamento antes de chamar `actor.run()`.
- A experiência observada é escrita no buffer de replay em cada passo de coleta de dados.

Conforme os Actors executam os passos de coleta de dados, eles passam trajetórias de (estado, ação, recompensa) ao observador, que as armazena em cache e escreve no sistema de replay Reverb.

Estamos armazenando trajetórias para frames [(t0,t1) (t1,t2) (t2,t3), ...] porque `stride_length=1`.

In [ ]:
rb_observer = reverb_utils.ReverbAddTrajectoryObserver(
  reverb_replay.py_client,
  table_name,
  sequence_length=2,
  stride_length=1)

Criamos um Actor com a política aleatória e coletamos experiências para semear o buffer de replay.

In [ ]:
initial_collect_actor = actor.Actor(
  collect_env,
  random_policy,
  train_step,
  steps_per_run=initial_collect_steps,
  observers=[rb_observer])
initial_collect_actor.run()

Instancie um Actor com a política de coleta para reunir mais experiências durante o treinamento.

In [ ]:
env_step_metric = py_metrics.EnvironmentSteps()
collect_actor = actor.Actor(
  collect_env,
  collect_policy,
  train_step,
  steps_per_run=1,
  metrics=actor.collect_metrics(10),
  summary_dir=os.path.join(tempdir, learner.TRAIN_DIR),
  observers=[rb_observer, env_step_metric])

Crie um Actor que será usado para avaliar a política durante o treinamento. Passamos `actor.eval_metrics(num_eval_episodes)` para registrar as métricas mais tarde.

In [ ]:
eval_actor = actor.Actor(
  eval_env,
  eval_policy,
  train_step,
  episodes_per_run=num_eval_episodes,
  metrics=actor.eval_metrics(num_eval_episodes),
  summary_dir=os.path.join(tempdir, 'eval'),
)

## Learners

O componente Learner contém o agente e realiza atualizações de passos de gradiente nas variáveis da política usando dados de experiências do buffer de replay. Após um ou mais passos de treinamento, o Learner pode enviar um novo conjunto de valores de variáveis para o container de variável.

In [ ]:
saved_model_dir = os.path.join(tempdir, learner.POLICY_SAVED_MODEL_DIR)

# Triggers to save the agent's policy checkpoints.
learning_triggers = [
    triggers.PolicySavedModelTrigger(
        saved_model_dir,
        tf_agent,
        train_step,
        interval=policy_save_interval),
    triggers.StepPerSecondLogTrigger(train_step, interval=1000),
]

agent_learner = learner.Learner(
  tempdir,
  train_step,
  tf_agent,
  experience_dataset_fn,
  triggers=learning_triggers,
  strategy=strategy)

## Métricas e avaliação

Instanciamos o Actor de avaliação com a `actor.eval_metrics` acima, que gera as métricas mais usadas durante a avaliação da política:

- Retorno médio. O retorno é a soma das recompensas obtidas ao executar uma política em um ambiente para um episódio e, geralmente, calculamos essa média em alguns episódios.
- Duração média do episódio.

Executamos o Actor para gerar essas métricas.

In [ ]:
def get_eval_metrics():
  eval_actor.run()
  results = {}
  for metric in eval_actor.metrics:
    results[metric.name] = metric.result()
  return results

metrics = get_eval_metrics()

In [ ]:
def log_eval_metrics(step, metrics):
  eval_results = (', ').join(
      '{} = {:.6f}'.format(name, result) for name, result in metrics.items())
  print('step = {0}: {1}'.format(step, eval_results))

log_eval_metrics(0, metrics)

Confira o [módulo de métricas](https://github.com/tensorflow/agents/blob/master/tf_agents/metrics/tf_metrics.py) para outras implementações padrão de métricas diferentes.

## Treinamento do agente

O loop de treinamento envolve tanto coletar os dados do ambiente quanto otimizar as redes do agente. Ao longo do caminho, vamos avaliar ocasionalmente a política do agente para ver como está o desempenho.

In [ ]:
#@test {"skip": true}
try:
  %%time
except:
  pass

# Reset the train step
tf_agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = get_eval_metrics()["AverageReturn"]
returns = [avg_return]

for _ in range(num_iterations):
  # Training.
  collect_actor.run()
  loss_info = agent_learner.run(iterations=1)

  # Evaluating.
  step = agent_learner.train_step_numpy

  if eval_interval and step % eval_interval == 0:
    metrics = get_eval_metrics()
    log_eval_metrics(step, metrics)
    returns.append(metrics["AverageReturn"])

  if log_interval and step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, loss_info.loss.numpy()))

rb_observer.close()
reverb_server.stop()

## Visualização


### Plots

Podemos plotar o retorno médio em comparação com os passos dos globais para ver o desempenho do nosso agente. No `Minitaur`, a função de recompensa é baseada na distância que o minitaur caminha em 1000 passos e penaliza o gasto de energia.

In [ ]:
#@test {"skip": true}

steps = range(0, num_iterations + 1, eval_interval)
plt.plot(steps, returns)
plt.ylabel('Average Return')
plt.xlabel('Step')
plt.ylim()

### Vídeos

É útil visualizar o desempenho de um agente ao renderizar o ambiente a cada passo. Antes de fazer isso, vamos criar uma função para incorporar vídeos neste colab.

In [ ]:
def embed_mp4(filename):
  """Embeds an mp4 file in the notebook."""
  video = open(filename,'rb').read()
  b64 = base64.b64encode(video)
  tag = '''
  <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
  Your browser does not support the video tag.
  </video>'''.format(b64.decode())

  return IPython.display.HTML(tag)

O código a seguir visualiza a política do agente por alguns episódios:

In [ ]:
num_episodes = 3
video_filename = 'sac_minitaur.mp4'
with imageio.get_writer(video_filename, fps=60) as video:
  for _ in range(num_episodes):
    time_step = eval_env.reset()
    video.append_data(eval_env.render())
    while not time_step.is_last():
      action_step = eval_actor.policy.action(time_step)
      time_step = eval_env.step(action_step.action)
      video.append_data(eval_env.render())

embed_mp4(video_filename)